<a href="https://colab.research.google.com/github/svishakan/Artificial-Intelligence/blob/master/AI%20Lab%20-%20Exam%20-%20CAT4%20-%20Paragraph%20Sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Question:- </h3>
Given n paragraphs numbered from 1 to n, arrange them in the order of 1, 2, . . . , n where n ≤ 9. With
the help of a clipboard, you can press Ctrl-X (cut) and Ctrl-V (paste) several times. You cannot cut twice
before pasting, but you can cut several contiguous paragraphs at the same time and these paragraphs will
later be pasted in order. What is the minimum number of steps required?
<br><br>Example 1: Make [2, 4, 1, 5, 3, 6] sorted.
<br>[2, 4, 1, 5, 3, 6] cut paragraph (1) and paste it before paragraph (2)
<br>[1, 2, 4, 5, 3, 6] cut paragraph (3) and paste it before paragraph (4)
<br>[1, 2, 3, 4, 5, 6] Minimum number of steps required = 2
<br><br>Example 2: Make [(3, 4, 5), 1, 2] sorted.
<br>[3, 4, 5, 1, 2] cut three paragraphs 3, 4, 5 at the same time and paste them after paragraph 2
<br>[1, 2, 3, 4, 5] Minimum number of steps required = 1
<br>This solution is not unique as we can have the following alternative answer:
[3, 4, 5, 1, 2] cut two paragraphs 1, 2 at the same time and paste them before paragraph 3
<br>[1, 2, 3, 4, 5] Minimum number of steps required = 1
<br><br>Example 3: A trivial algorithm will process [5, 4, 3, 2, 1] as follows: [(5), 4, 3, 2, 1] → [(4), 3, 2, 1, 5] →
[(3), 2, 1, 4, 5] → [(2), 1, 3, 4, 5] → [1, 2, 3, 4, 5] of total 4 cut-paste steps. 
<br>This is not optimal, as we can
solve this instance in only 3 steps: [5, 4,(3, 2), 1] → [3,(2, 5), 4, 1] → [3, 4,(1, 2), 5] → [1, 2, 3, 4, 5].
<br><br><br>
1. Formulate the problem as a state-space search problem. Implement a program to solve the problem
using best-first search algorithm. You can use suitable data structures available in Python. (25)
(a) For each function, providee code for testing your function (Program with no testing code will
get lesser points).
<br><br>
2. Modify your program to solve the problem using A* algorithm. Decide a suitable heuristic for the
problem. (25)
<br><br>
(a) For each function, provide code for testing your function (Program with no testing code will
get lesser points).
(b) Give a measure of how much A* performs better than best-first search.
<br>

In [ ]:
"""
Name:               S. Vishakan
Register Number:    18 5001 196
Class:              CSE - C
"""

In [2]:
from collections import deque
import heapq
import math
import random
import time

In [3]:
class Sorter:
    """Class to encapsulate the problem state. """
    
    def __init__(self, state, parent = None, depth = 0, is_back_cost = False):
        self.n = len(state)
        self.state = state
        self.cost = self.heuristic()
        self.depth = depth
        self.is_back_cost = is_back_cost
        self.parent = parent

    def __str__(self):
        return str(self.state)

    def __lt__(self, other_state):
        if not self.is_back_cost:
            #A-star
            return self.cost < other_state.cost 
        #BFS
        return self.cost + self.depth < other_state.cost + other_state.depth

    def heuristic(self):
        cost = 0

        for i in range(1, self.n):
            if self.state[i] != self.state[i-1] + 1:
                cost += 1
            
            if self.state[0] != 1:
                cost += 1

        return cost

    def is_goal(self):
        goal = sorted(self.state)

        if goal == self.state:
            return True

        return False

    def find_neighbors(self):
        next_states = []

        for left in range(self.n):
            for right in range(left+1, self.n+1):
                sublist = self.state[left:right]
                oldlist = [x for x in self.state]

                for i in range(right - 1, left - 1, -1):
                    oldlist.pop(i)
                
                for i in range(len(oldlist) + 1):
                    newlist = oldlist[:i] + sublist + oldlist[i:]

                    if newlist != self.state and newlist not in next_states:
                        next_states.append(newlist)
                    
        for i, state in enumerate(next_states):
            next_states[i] = Sorter(state, self, self.is_back_cost, self.depth + 1)


        random.shuffle(next_states)
        return next_states

In [4]:
class Solver:
    """Class to solve a given problem state. """
    
    def __init__(self):
        self.solution = None
        self.steps = 0

    def a_star(self, initial):

        frontier = []
        explored = set()
        visited = set()

        heapq.heappush(frontier, initial)
        visited.add(tuple(initial.state))

        while frontier:
            current = heapq.heappop(frontier)
            
            if tuple(current.state) in explored:
                continue

            explored.add(tuple(current.state))

            if current.is_goal():
                self.solution = current
                return self.solution, self.steps
            
            else:
                self.steps += 1
            
            for next_state in current.find_neighbors():
                if tuple(next_state.state) not in visited:
                    heapq.heappush(frontier, next_state)
                    visited.add(tuple(next_state.state))

        return None, None

    def bfs(self, initial):

        frontier = []
        explored = set()
        visited = set()

        heapq.heappush(frontier, initial)
        visited.add(tuple(initial.state))

        while frontier:
            current = heapq.heappop(frontier)

            if tuple(current.state) in explored:
                continue

            explored.add(tuple(current.state))

            if current.is_goal():
                self.solution = current
                return self.solution, self.steps
            
            else:
                self.steps += 1

            for next_state in current.find_neighbors():
                if tuple(next_state.state) not in visited:
                    visited.add(tuple(next_state.state))
                    heapq.heappush(frontier, next_state)

        return None, None

    def get_steps(self):
        current = self.solution
        path = []

        while current != None:
            path.append(current)
            current = current.parent
        
        return path[::-1]


In [5]:
array1 = [2, 4, 1, 5, 3, 6]
array2 = [3, 4, 5, 1, 2]
array3 = [5, 4, 3, 2, 1]

In [6]:
print("\n\t\tBest First Search\n")

start = time.time()

initial = Sorter(array1)
solver = Solver()
soln, steps = solver.bfs(initial)

end = time.time()

if soln:
    print("Initial State:", initial)
    print("Solution Reached:", soln)
    print("No. of steps taken:", steps)

    print("\nPath taken:\n")
    path = solver.get_steps()
    
    for state in path:
        print(state)

    print("\nTime taken: %.5f"%(end - start))

else:
    print("No solution.")


		Best First Search

Initial State: [2, 4, 1, 5, 3, 6]
Solution Reached: [1, 2, 3, 4, 5, 6]
No. of steps taken: 2

Path taken:

[2, 4, 1, 5, 3, 6]
[1, 2, 4, 5, 3, 6]
[1, 2, 3, 4, 5, 6]

Time taken: 0.00114


In [7]:
print("\n\t\tA - Star Search\n")

start = time.time()

initial = Sorter(array1, is_back_cost = True)
solver2 = Solver()
soln2, steps2 = solver2.a_star(initial)

end = time.time()

if soln:
    print("Initial State:", initial)
    print("Solution Reached:", soln2)
    print("No. of steps taken:", steps2)

    print("\nPath taken:\n")
    path2 = solver2.get_steps()
    
    for state in path2:
        print(state)
    
    print("\nTime taken: %.5f"%(end - start))

else:
    print("No solution.")


		A - Star Search

Initial State: [2, 4, 1, 5, 3, 6]
Solution Reached: [1, 2, 3, 4, 5, 6]
No. of steps taken: 2

Path taken:

[2, 4, 1, 5, 3, 6]
[1, 2, 4, 5, 3, 6]
[1, 2, 3, 4, 5, 6]

Time taken: 0.00123


In [8]:
print("\n\t\tBest First Search\n")

start = time.time()

initial = Sorter(array2)
solver = Solver()
soln, steps = solver.bfs(initial)

end = time.time()

if soln:
    print("Initial State:", initial)
    print("Solution Reached:", soln)
    print("No. of steps taken:", steps)

    print("\nPath taken:\n")
    path = solver.get_steps()
    
    for state in path:
        print(state)

    print("\nTime taken: %.5f"%(end - start))

else:
    print("No solution.")


		Best First Search

Initial State: [3, 4, 5, 1, 2]
Solution Reached: [1, 2, 3, 4, 5]
No. of steps taken: 1

Path taken:

[3, 4, 5, 1, 2]
[1, 2, 3, 4, 5]

Time taken: 0.00026


In [9]:
print("\n\t\tA - Star Search\n")

start = time.time()

initial = Sorter(array2, is_back_cost = True)
solver2 = Solver()
soln2, steps2 = solver2.a_star(initial)

end = time.time()

if soln:
    print("Initial State:", initial)
    print("Solution Reached:", soln2)
    print("No. of steps taken:", steps2)

    print("\nPath taken:\n")
    path2 = solver2.get_steps()
    
    for state in path2:
        print(state)
    
    print("\nTime taken: %.5f"%(end - start))

else:
    print("No solution.")


		A - Star Search

Initial State: [3, 4, 5, 1, 2]
Solution Reached: [1, 2, 3, 4, 5]
No. of steps taken: 1

Path taken:

[3, 4, 5, 1, 2]
[1, 2, 3, 4, 5]

Time taken: 0.00027


In [10]:
#Thus A-Star takes more time than BFS 
#but the difference is almost negligible due to the small state space of the problem